# Gulf Migratory Fish Connectivity

This indicator was used in South Atlantic 2021 Blueprint.

Originally, we were using it for both the Atlantic and the Gulf drainages. When we found the Atlantic migratory fish analysis from SARP, we modified this to only apply to the Gulf drainages.


Created by Amy Keister, last run by Amy Keister on 17 May 2023. It took 10 minutes to run

In [44]:
import os
import arcpy

In [45]:
import time
start = time.time()

In [46]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace= r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\GulfMigratoryFish\GulfMigratoryFishCo.gdb"

In [53]:
# define final indicator outputs
Out = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\GulfMigratoryFish\GulfMigratoryFishConnectivity.tif"

In [48]:
# define rasters used for cell size, extent, and snapping
Rextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [49]:
# define inputs
# species layers from the SEACAP project
amshad="F:\\GIS_DATA\\WaterResources\\SEACAP\\SEACAP_forSALCC.gdb\\NAD83_Albers\\amshad1"
alshad="F:\\GIS_DATA\\WaterResources\\SEACAP\\SEACAP_forSALCC.gdb\\NAD83_Albers\\alashad1"
#blueback="F:\\GIS_DATA\\WaterResources\\SEACAP\\SEACAP_forSALCC.gdb\\NAD83_Albers\\blueback1"
strbass="F:\\GIS_DATA\\WaterResources\\SEACAP\\SEACAP_forSALCC.gdb\\NAD83_Albers\\strbass1"
#astur="F:\\GIS_DATA\\WaterResources\\SEACAP\\SEACAP_forSALCC.gdb\\NAD83_Albers\\atlstur1"
gstur="F:\\GIS_DATA\\WaterResources\\SEACAP\\SEACAP_forSALCC.gdb\\NAD83_Albers\\gulflstur1"
# critical habitat for Sturgeon from NOAA
# asturCH= r"F:\GIS_DATA\SpeciesAndHabitats\NOAA_CriticalHabitat\shapefile-Atlantic-sturgeon-critical-habitat-GARFO-SERO\shapefile-Atlantic-sturgeon-critical-habitat-GARFO-SERO.shp"
gsturCH= r"F:\GIS_DATA\SpeciesAndHabitats\NOAA_CriticalHabitat\shapefile-Gulf-sturgeon-critical-habitat-all-units-USFWS-NMFS-SERO\shapefile_Gulf_sturgeon_critical_habitat_Unit1_7_USFWS_SERO.shp"
gsturCHp= r"F:\GIS_DATA\SpeciesAndHabitats\NOAA_CriticalHabitat\shapefile-Gulf-sturgeon-critical-habitat-all-units-USFWS-NMFS-SERO\shapefile_Gulf_sturgeon_critical_habitat_Unit8_14_SERO.shp"
# EPA floodplain
FP = r"F:\GIS_DATA\WaterResources\Estimated_floodplain_CONUS\Estimated_floodplain_CONUS.tif"
# HUCs
H12 = r"F:\GIS_DATA\WaterResources\NHD\WBD_National_GDB\WBD_National_GDB.gdb\WBD\WBDHU12"
H6= r"F:\GIS_DATA\WaterResources\NHD\WBD_National_GDB\WBD_National_GDB.gdb\WBD\WBDHU6"

### Start Analysis

In [50]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [51]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\GulfMigratoryFish\GulfMigratoryFishCo.gdb


### Assign species line work to an area 

These steps use the HUC12s 

I tried using the high resolution catchments, but they left out areas inside large floodplains that did not intersect with the lines.

I tried using the NHDPlus medium resolution catchments to represent locations where species have either been observed, or where there is critical habitat for those species, but they didn't do much better than the high res on the large river floodplains.

I also tried using the size classes, but many large floodplains still have small size classes in the center of the floodplains.

I tried using the HUC12s to slice up the functional networks, but that resulted in holes I had to fill in in convoluted ways

I join the non sturgeon species and the sturgeon species seperately so I can make sure that for lines that contain both, the sturgeon values take precedence

In [9]:
# make a copy of the gulf sturgeon critical habiat lines for edits
arcpy.management.CopyFeatures(gsturCH, "gsturCH1", '', None, None, None)

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\GulfMigratoryFish\\GulfMigratoryFishCo.gdb\\gsturCH1'>

In [10]:
# delete all fields from the copy of the gulf sturgeon
fc = "gsturCH1"
for f in arcpy.ListFields(fc):
  if (f.name == 'OBJECTID' or f.name == 'Shape' or f.name == 'Shape_Length'):
    print("can't delete {}".format(f.name))
  else:
    arcpy.management.DeleteField(fc, f.name)

can't delete OBJECTID
can't delete Shape
can't delete Shape_Length


In [11]:
# add a field to gulf sturgeon critical habitat and calculate it
arcpy.management.CalculateField(gsturCH, "gsturCH1", "1", "PYTHON3", '', "SHORT")

<Result 'F:\\GIS_DATA\\SpeciesAndHabitats\\NOAA_CriticalHabitat\\shapefile-Gulf-sturgeon-critical-habitat-all-units-USFWS-NMFS-SERO\\shapefile_Gulf_sturgeon_critical_habitat_Unit1_7_USFWS_SERO.shp'>

In [12]:
# merge all no sturgeon species linework into a single layer with attributes for each species
# don't use this b/c bluepback aren't in gulf
#arcpy.management.Merge([amshad, alshad, blueback, strbass], "MSpeciesLines3", "", "ADD_SOURCE_INFO")

In [13]:
# merge all no sturgeon species linework into a single layer with attributes for each species
arcpy.management.Merge([amshad, alshad, strbass], "MSpeciesLines3", "", "ADD_SOURCE_INFO")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\GulfMigratoryFish\\GulfMigratoryFishCo.gdb\\MSpeciesLines3'>

In [14]:
# add and calculate a field to show which lines are associated with the non sturgen species
arcpy.management.CalculateField("MSpeciesLines3", "NotSturg", '"NotSturg"', "PYTHON3", '', "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\GulfMigratoryFish\\GulfMigratoryFishCo.gdb\\MSpeciesLines3'>

In [15]:
# merge all sturgeon species linework into a single layer with attributes for each species
# don't use Atlantic sturgeon lines since we are restricting this to the gulf
# arcpy.management.Merge([astur, gstur, "asturCH1", "gsturCH1"], "MSpeciesLines4", "", "ADD_SOURCE_INFO")

In [16]:
# merge all gulf sturgeon species linework into a single layer with attributes for each species
arcpy.management.Merge([gstur, "gsturCH1"], "MSpeciesLines4", "", "ADD_SOURCE_INFO")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\GulfMigratoryFish\\GulfMigratoryFishCo.gdb\\MSpeciesLines4'>

In [17]:
# add and calculate a field to show which lines are associated with gulf sturgeon species
arcpy.management.CalculateField("MSpeciesLines4", "Sturg", '"Sturg"', "PYTHON3", '', "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\GulfMigratoryFish\\GulfMigratoryFishCo.gdb\\MSpeciesLines4'>

### Join species lines to HUC12

In [18]:
# make a copy of the HUC12 for edits
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.management.CopyFeatures(H12, "H12", '', None, None, None)

In [19]:
# perform a spatial join with the merged line work to huc12s 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.analysis.SpatialJoin("H12", "MSpeciesLines4", "poly4", "JOIN_ONE_TO_ONE", "KEEP_ALL")

In [20]:
# perform a spatial join with the merged line work to huc12s
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.analysis.SpatialJoin("poly4", "MSpeciesLines3", "poly43", "JOIN_ONE_TO_ONE", "KEEP_ALL")

In [21]:
# set code block for next step
codeblock = """
def Reclass(Sturg, NotSturg):
    if (Sturg == "Sturg"):
        return 2
    elif (NotSturg == "NotSturg"):
        return 1  
"""

In [22]:
# add and calculate a field of the HUC12 that intersect the species linework to contain
# values of 1 or 2 based on species presence
arcpy.management.CalculateField("poly43", "Value", "Reclass(!Sturg!,!NotSturg!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\GulfMigratoryFish\\GulfMigratoryFishCo.gdb\\poly43'>

In [23]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.FeatureToRaster(in_features="poly43", field="Value", out_raster="poly43r", cell_size=Rextent)

### clip to floodplains

I am clipping to floodplains before I add in the polygon critical habitat for the gulf sturgeon, because that extents out into the near shore environment and our floodplain clip was removing it.

In [24]:
# Limit the result to the EPA floodplain
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent):
    out_raster = arcpy.sa.Con(FP, "poly43r", '',"Value = 1"); out_raster.save("poly43rFP")

In [25]:
# If we want places outside the floodplain to be 0, we can use this one
# out_raster = arcpy.sa.Con(FP, "indicator2", '0',"Value = 1"); out_raster.save("poly43rFP")

### add sturgeon polygon data

In [26]:
# make a copy of the gulf sturgeon polygons for edits
arcpy.management.CopyFeatures(gsturCHp, "gsturCHp1", '', None, None, None)

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\GulfMigratoryFish\\GulfMigratoryFishCo.gdb\\gsturCHp1'>

In [27]:
# add a field to gulf sturgeon critical habitat polygon and give them a value of 2
arcpy.management.CalculateField("gsturCHp1", "gsturCHp1", "2", "PYTHON3", '', "SHORT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\GulfMigratoryFish\\GulfMigratoryFishCo.gdb\\gsturCHp1'>

In [28]:
# convert gulf sturgeon critical habitat polygon to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent):
    arcpy.conversion.FeatureToRaster("gsturCHp1", "gsturCHp1", "gsturCHpR", cell_size=Rextent)

In [29]:
# use cell statistics to combine the raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent):
    out_raster = arcpy.sa.CellStatistics("poly43rFP;gsturCHpR", "MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("allSpecies")

### clip to gulf

In [30]:
# make a copy of the HUC6 for edits
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.management.CopyFeatures(H6, "H6", '', None, None, None)

In [31]:
# select the Gulf drainage hucs that intersects the input data
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.analysis.Select("H6", "H6_Select", "huc6 IN ('031700','031602','031800','080902','031501', '031403', '031200', '031300', '031101', '031502', '031401', '031402', '031102')")

In [32]:
# add and calculate a field so I can dissolve the SA Gulf Huc6
arcpy.management.CalculateField("H6_Select", "dissolve", '0', "PYTHON3", '', "SHORT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\GulfMigratoryFish\\GulfMigratoryFishCo.gdb\\H6_Select'>

In [33]:
arcpy.management.Dissolve("H6_Select", "H6_SelectDiss", "dissolve", None, "MULTI_PART", "DISSOLVE_LINES")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\GulfMigratoryFish\\GulfMigratoryFishCo.gdb\\H6_SelectDiss'>

In [34]:
# remove areas outside the Gulf drainages where the input data apply
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent):
    out_raster = arcpy.sa.ExtractByMask("allSpecies", "H6_SelectDiss"); out_raster.save("allSpeciesG")

### add zero values

In [35]:
# convert the gulf HUC6s where we have data to a raster, this will be used to make a study area
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.FeatureToRaster(in_features="H6_SelectDiss", field="dissolve", out_raster="H6_SelectDissR", cell_size=Rextent)

In [36]:
#use cell statistics max to combine above rasters
out_raster = arcpy.sa.CellStatistics("allSpeciesG;H6_SelectDissR", "MAXIMUM", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [54]:
# clip to Base Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("SEMask")

In [55]:
# export as .tif with Base Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("SEMask", Out, '', None, "15", "NONE", "NONE", "4_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [56]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 2:
        return '2 = Presence of Gulf sturgeon'
    elif value == 1:
        return '1 = Presence of Alabama shad, American shad, or striped bass' 
    elif value == 0:
        return '0 = Not identified as Gulf migratory fish habitat (east of the Mississippi River)'
"""

In [57]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(Out, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\GulfMigratoryFish\\GulfMigratoryFishConnectivity2023.tif'>

In [58]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 2:
		return 49
	if Value == 1:
		return 158
	else:
		return 255
		
def Reclass2(Value):
	if Value == 2:
		return 130
	if Value == 1:
		return 202
	else:
		return 255
		
def Reclass3(Value):
	if Value == 2:
		return 189
	if Value == 1:
		return 225
	else:
		return 255
"""

In [59]:
# calculate Red field
arcpy.management.CalculateField(Out, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(Out, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(Out, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\GulfMigratoryFish\\GulfMigratoryFishConnectivity2023.tif'>

In [43]:
# this prints the time it took this notebook to run in seconds
end = time.time()
print(end - start)

673.2844398021698
